In [1]:
import json
path = 'E:\Python For Data Analysis @220921\data\\usa_bitly.txt'
records = [json.loads(line) for line in open(path)]


In [2]:
#extract a list of time zones using a list comprehension:
time_zones = [rec['tz'] for rec in records if 'tz' in rec]
time_zones;

In [3]:
#produce counts of time zones using dict() - full list
def get_counts_dict(sequence):
    counts = {}  
    for x in sequence:
        if x in counts:
            counts[x] +=1
        else:
            counts[x] = 1         
    return counts

get_counts_dict(time_zones)

{'America/New_York': 1251,
 'America/Denver': 191,
 'America/Sao_Paulo': 33,
 'Europe/Warsaw': 16,
 '': 521,
 'America/Los_Angeles': 382,
 'Asia/Hong_Kong': 10,
 'Europe/Rome': 27,
 'Africa/Ceuta': 2,
 'Europe/Madrid': 35,
 'Asia/Kuala_Lumpur': 3,
 'Asia/Nicosia': 1,
 'Europe/London': 74,
 'Pacific/Honolulu': 36,
 'America/Chicago': 400,
 'Europe/Malta': 2,
 'Europe/Lisbon': 8,
 'Europe/Paris': 14,
 'Europe/Copenhagen': 5,
 'America/Mazatlan': 1,
 'Europe/Dublin': 3,
 'Europe/Brussels': 4,
 'America/Vancouver': 12,
 'Europe/Amsterdam': 22,
 'Europe/Prague': 10,
 'Europe/Stockholm': 14,
 'America/Anchorage': 5,
 'Asia/Bangkok': 6,
 'Europe/Berlin': 28,
 'America/Rainy_River': 25,
 'Europe/Budapest': 5,
 'Asia/Tokyo': 37,
 'Europe/Vienna': 6,
 'America/Phoenix': 20,
 'Asia/Jerusalem': 3,
 'Asia/Karachi': 3,
 'America/Bogota': 3,
 'America/Indianapolis': 20,
 'America/Montreal': 9,
 'Asia/Calcutta': 9,
 'Europe/Skopje': 1,
 'Asia/Beirut': 4,
 'Australia/NSW': 6,
 'Chile/Continental': 6,
 

In [4]:
#produce counts of time zones using dict() - one key:value pair of dict
def get_counts_one(sequence):
    counts = {}
    for x in sequence:
        if x in counts:
            counts[x] +=1
        else:
            counts[x]=1
        return counts
    
get_counts_one(time_zones)

{'America/New_York': 1}

In [5]:
#produce counts of time zones using python standard library

from collections import defaultdict
def get_counts_psl(sequence):
    counts = defaultdict(int)  
    for x in sequence:
        counts[x]+=1
        
    return counts

get_counts_psl(time_zones);

In [6]:
#get counts of particular column value - function call
counts_psl = get_counts_psl(time_zones)
counts['America/New_York']

counts_dict = get_counts_dict(time_zones)
counts_dict

counts_one = get_counts_one(time_zones)
counts_one

NameError: name 'counts' is not defined

In [ ]:
#calculate length of time_zone
len(time_zones)

In [ ]:
# top 10 time zones and their counts -  not working
def top_counts(count_dict, n=10):
    value_key_pairs = [(count, tz) for tz, count in count_dict.items()]
    value_key_pairs.sort()
    return value_key_pairs[-n:]

# top 10 time zones and their counts -  using python standard library
from collections import Counter
counts = Counter(time_zones)
counts.most_common(10)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
frame = pd.DataFrame(records)
frame.count
frame['tz'][:10]

#time_zones - missing value treatment
clean_tz = frame['tz'].fillna('Missing')

#time_zones - space cleaning
clean_tz[clean_tz == ''] = 'Unknown'
count_tz = clean_tz.value_counts()
count_tz[:10]

#Draw Plot - Default
count_tz[:10].plot(kind='bar', rot=0, figsize=(10,6))


In [ ]:
#access field contains information about the browser, device or application used to perform the URL shortening:
frame['a'][1]
frame['a'][50]

In [ ]:
from pandas import Series
#split off the token/tokens in the string (corresponding roughly to the browser capability) and make 
#another summary of the user behavior:
results = Series([x.split() for x in frame.a.dropna()]) #strings in column 'a'
results = Series([x.split()[0] for x in frame.a.dropna()])  #first token of string in column 'a'
results = Series([x.split()[:5] for x in frame.a.dropna()]) #first one to 4 token of string
results[:10];

#counts the values of results
result_count = Series([x.split()[0] for x in frame.a.dropna()])
result_count.value_counts();

#decompose the top time zones into Windows and nonWindows users 
import numpy as np
notnull_frame = frame[frame.a.notnull()]  #removing Nan from column 'a'
operating_system = np.where(notnull_frame['a'].str.contains('Windows'), 'Windows ', 'Not Windows ') # compute a value whether each row is Windows or not:
by_tz_os = notnull_frame.groupby(['tz', operating_system]); # group the data by its time zone column

#group counts is analogous to the value_counts that can be computed using size() 
#followed by reshaping into table with unstack()
agg_count = by_tz_os.size().unstack().fillna(0)
agg_count;

indexer = agg_count.sum(1).argsort() #indirect index array from the row counts in agg_counts then sort:
count_subset = agg_count.take(indexer)[-10:] #take to select the rows in that order,
count_subset.plot(kind='barh', rot=0, stacked=True)

#The plot doesn’t make it easy to see the relative percentage of Windows users in the smaller groups, but the rows can 
#easily be normalized to sum to 1 then plotted again
normed_subset = count_subset.div(count_subset.sum(1), axis=0)
normed_subset.plot(kind='barh', stacked=True)
